https://www.janestreet.com/puzzles/die-agony-index/

In [1]:
import numpy as np
import copy

# Defined in question

In [2]:
GRID = [
    [57, 33, 132, 268, 492, 732],
    [81, 123, 240, 443, 353, 508],
    [186, 42, 195, 704, 452, 228],
    [-7, 2, 357, 452, 317, 395],
    [5, 23, -4, 592, 445, 620],
    [0, 77, 32, 403, 337, 452]
][::-1]
GRID

[[0, 77, 32, 403, 337, 452],
 [5, 23, -4, 592, 445, 620],
 [-7, 2, 357, 452, 317, 395],
 [186, 42, 195, 704, 452, 228],
 [81, 123, 240, 443, 353, 508],
 [57, 33, 132, 268, 492, 732]]

# Die class and recurssion function

In [3]:
class Die:
    '''
    The die is as follows, numbers are the indices of the list, 5 faces down and 0 faces up
    X1X
    254
    X3X
    X0X
    '''
    
    def __init__(self):
        self.faces = [np.nan] * 6
        
    def set_face(self, index, number):
        self.faces[index] = number
        
    def get_face(self, index):
        return self.faces[index]
        
    def tip(self, x_dir, y_dir):
        '''
        x_dir: right (1), left(-1), else (0)
        y_dir: forward (1), backwards (-1), else (0)
        '''
        duplicate = copy.deepcopy(self.faces)
        if x_dir == 1: # 0->4->5->2->0
            duplicate[4] = self.faces[0]
            duplicate[5] = self.faces[4]
            duplicate[2] = self.faces[5]
            duplicate[0] = self.faces[2]
        elif x_dir == -1: # 0->2->5->4->0
            duplicate[2] = self.faces[0]
            duplicate[5] = self.faces[2]
            duplicate[4] = self.faces[5]
            duplicate[0] = self.faces[4]
        elif x_dir != 0:
            raise Exception('Parameter x_dir invalid')
        if y_dir == 1: # 1->5->3->0->1
            duplicate[5] = self.faces[1]
            duplicate[3] = self.faces[5]
            duplicate[0] = self.faces[3]
            duplicate[1] = self.faces[0]
        elif y_dir == -1: # 1->0->3->5->1
            duplicate[0] = self.faces[1]
            duplicate[3] = self.faces[0]
            duplicate[5] = self.faces[3]
            duplicate[1] = self.faces[5]
        elif y_dir != 0:
            raise Exception('Parameter x_dir invalid')
        self.faces = duplicate

In [4]:
def simulate(x_pos, y_pos, die, count, score, all_xy):
    print(count, die.faces, x_pos, y_pos, all_xy)
    
    if np.isnan(die.get_face(0)):
        # set face if number is unassigned, assume face must be integer
        new_face = (GRID[y_pos][x_pos] - score)/count
        if int(new_face) == new_face:
            die.set_face(0, new_face)
        else:
            return False, [], die
    else:
        # terminate if score is wrong
        if die.get_face(0) * count + score != GRID[y_pos][x_pos]:
            return False, [], die
        
    # terminate if end point is reached
    if x_pos == 5 and y_pos == 5:
        return True, all_xy+[[x_pos, y_pos]], die
    
    can_forward, can_backward, can_right, can_left = [True]*4
    if x_pos == 0:
        can_left = False
    elif x_pos == 5:
        can_right = False
    if y_pos == 0:
        can_backward = False
    elif y_pos == 5:
        can_forward = False
    
    is_success = False
    if can_left:
        duplicated_die = copy.deepcopy(die)
        duplicated_die.tip(-1, 0)
        is_success, result, solution_die = simulate(
            x_pos - 1, y_pos, duplicated_die, count+1, GRID[y_pos][x_pos], all_xy+[[x_pos, y_pos]]
        )
    if not is_success and can_right:
        duplicated_die = copy.deepcopy(die)
        duplicated_die.tip(1, 0)
        is_success, result, solution_die = simulate(
            x_pos + 1, y_pos, duplicated_die, count+1, GRID[y_pos][x_pos], all_xy+[[x_pos, y_pos]]
        )
    if not is_success and can_forward:
        duplicated_die = copy.deepcopy(die)
        duplicated_die.tip(0, 1)
        is_success, result, solution_die = simulate(
            x_pos, y_pos + 1, duplicated_die, count+1, GRID[y_pos][x_pos], all_xy+[[x_pos, y_pos]]
        )
    if not is_success and can_backward:
        duplicated_die = copy.deepcopy(die)
        duplicated_die.tip(0, -1)
        is_success, result, solution_die = simulate(
            x_pos, y_pos - 1, duplicated_die, count+1, GRID[y_pos][x_pos], all_xy+[[x_pos, y_pos]]
        )
        
    return is_success, result, solution_die

# Get solution

In [5]:
die = Die()
for i in range(6):
    die.set_face(i, i)
die.faces

[0, 1, 2, 3, 4, 5]

In [6]:
is_success, result, solution_die = simulate(0, 1, Die(), 1, 0, [])

1 [nan, nan, nan, nan, nan, nan] 0 1 []
2 [nan, nan, nan, nan, 5.0, nan] 1 1 [[0, 1]]
3 [5.0, nan, 9.0, nan, nan, nan] 0 1 [[0, 1], [1, 1]]
3 [nan, nan, nan, nan, 9.0, 5.0] 2 1 [[0, 1], [1, 1]]
4 [9.0, nan, -9.0, nan, 5.0, nan] 1 1 [[0, 1], [1, 1], [2, 1]]
4 [nan, nan, 5.0, nan, -9.0, 9.0] 3 1 [[0, 1], [1, 1], [2, 1]]
5 [-9.0, nan, 149.0, nan, 9.0, 5.0] 2 1 [[0, 1], [1, 1], [2, 1], [3, 1]]
5 [5.0, nan, 9.0, nan, 149.0, -9.0] 4 1 [[0, 1], [1, 1], [2, 1], [3, 1]]
5 [nan, 149.0, 5.0, 9.0, -9.0, nan] 3 2 [[0, 1], [1, 1], [2, 1], [3, 1]]
6 [-9.0, 149.0, -28.0, 9.0, nan, 5.0] 2 2 [[0, 1], [1, 1], [2, 1], [3, 1], [3, 2]]
6 [5.0, 149.0, nan, 9.0, -28.0, -9.0] 4 2 [[0, 1], [1, 1], [2, 1], [3, 1], [3, 2]]
6 [9.0, -28.0, 5.0, nan, -9.0, 149.0] 3 3 [[0, 1], [1, 1], [2, 1], [3, 1], [3, 2]]
6 [149.0, nan, 5.0, -28.0, -9.0, 9.0] 3 1 [[0, 1], [1, 1], [2, 1], [3, 1], [3, 2]]
5 [nan, 9.0, 5.0, 149.0, -9.0, nan] 3 0 [[0, 1], [1, 1], [2, 1], [3, 1]]
4 [nan, -9.0, nan, 5.0, 9.0, nan] 2 2 [[0, 1], [1, 1], [

In [7]:
res = []
for r in result:
    if not r in res:
        res.append(r)
np.sum(GRID) - sum([GRID[r[1]][r[0]] for r in res])

1935